In [ ]:
%pip install pandas
%pip install numpy

In [13]:
import pandas as pd
file = open("..\peni\Left\peni_LeftHand_Coordinates.csv", 'r')
df = pd.read_csv(file)
file.close()

In [20]:
np_coords = df.to_numpy()
list_coords = list(np_coords)

In [49]:
parsed_list = []
for i in range(len(list_coords[0])):
    parsed_list.append(str(list_coords[0][i]).strip('()').split(','))

In [53]:
import numpy as np
parsed_array = np.array(parsed_list[0:21], dtype = np.float32)

In [55]:
from itertools import combinations
landmarks = np.arange(21)
combos = np.array(list(combinations(landmarks, 3)))
def joint_angles_fast(coords, combos):
    vecA = np.subtract(coords[combos[:, 1], :], coords[combos[:, 0], :])
    vecB = np.subtract(coords[combos[:, 1], :], coords[combos[:, 2], :])
    X = np.einsum('ij, ij -> i', vecA, vecB)/np.linalg.norm(vecA, 2, 1)/np.linalg.norm(vecB, 2, 1)
    Y = np.linalg.norm(np.cross(vecA, vecB), 2, 1)/np.linalg.norm(vecA, 2, 1)/np.linalg.norm(vecB, 2, 1)
    angles = np.arctan2(Y, X)
    return angles

In [58]:
true_angles = joint_angles_fast(parsed_array, combos)

In [73]:
#Attempt to do reconstruction using only the angles, using my hand as a starting point
#We know the angles for sure

# By rodrigues angle formula:
for j in range(len(combos)):
    i = 1329 - j
    vecA = np.subtract(parsed_array[combos[i, 1], :], parsed_array[combos[i, 0], :])
    vecB = np.subtract(parsed_array[combos[i, 1], :], parsed_array[combos[i, 2], :])
    theta = true_angles[i]
    k = vecA/np.linalg.norm(vecA)
    vecB_new = vecB * np.cos(theta) + np.cross(k, vecB) * np.sin(theta) + k * np.dot(k, vecB)* (1-np.cos(theta))
    parsed_array[combos[i, 2], :] = parsed_array[combos[i, 1], :] + vecB

In [74]:
joint_angles_fast(parsed_array, combos)

array([2.7042944, 2.6133666, 2.5968156, ..., 3.1004274, 3.1008115,
       3.1187978], dtype=float32)

In [76]:
true_angles

array([2.7042944, 2.6133666, 2.5968156, ..., 3.1004274, 3.1008115,
       3.1187978], dtype=float32)

In [78]:
diff = true_angles - (joint_angles_fast(parsed_array, combos))